In [1]:
import pandas as pd
from pandas import array
from pandas import DataFrame

import pickle

import numpy as np
from numpy import zeros, array

from rdkit import Chem, DataStructs
from rdkit.Chem import Draw, Descriptors, AllChem
import rdkit.Chem.AllChem as AllChem

from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score, mean_squared_error, balanced_accuracy_score
from sklearn.model_selection import KFold, cross_val_predict, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

from scopy.ScoTox import Toxfilter

from itertools import groupby

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
data = pd.read_csv("tox_set.csv", sep='\t', header=None)

In [3]:
data.columns = ['SMILES', 'ID', 'CLASS']

In [4]:
data

,SMILES,ID,CLASS
0,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,DB00014,0
1,N=C(N)NCCC[C@H](NC(=O)[C@@H]1CCCN1C(=O)[C@H]1C...,DB00035,0
2,CC(=O)N[C@H](Cc1ccc2ccccc2c1)C(=O)N[C@H](Cc1cc...,DB00050,0
3,C/C=C/C[C@@H](C)[C@H](O)[C@@H]1/C(O)=N/[C@@H](...,DB00091,0
4,NCCCC[C@H](NC(=O)[C@@H]1CCCN1C(=O)[C@H]1CSSC[C...,DB00093,0
...,...,...,...
8942,C[C@@H](N)NCCO,ZINC98359253,1
8943,O=P(O)(O)OC(c1ccccc1)c1ccccc1,ZINC98359257,1
8944,CCCCO[C@@H](C)OC[C@@H](C)O,ZINC98359263,1
8945,C=C(CCN(CC)CC)C(=O)O,ZINC98359322,1


In [5]:
smiles = data['SMILES'].to_list()
labels = data['CLASS'].to_list()

In [6]:
data['CLASS'].value_counts()

0    4893
1    4054
Name: CLASS, dtype: int64

In [7]:
mols = []

for smile in smiles:
    mol = Chem.MolFromSmiles(smile)
    if mol is not None:
        mols.append(mol)
        
print(len(mols))
print(len(smiles))

8947
8947


Посчитаем дополнительные дескрипторы

In [8]:
Filter = Toxfilter(mols, detail=True, showSMILES=True)
filter_result = Filter.Check_Toxicophores()
filter_result_groupby = groupby(sorted(filter_result, key=lambda x: x['Disposed']), lambda x: x['Disposed'])
for k, g in filter_result_groupby:
     print(k, len(list(g)))

Accepted 2897
Rejected 6050


In [9]:
disposed = []

for fr in filter_result:
    disposed.append(filter_result[filter_result.index(fr)]['Disposed'])

print(len(disposed))

8947


In [10]:
for d in disposed:
    if d == 'Rejected':
        disposed[disposed.index(d)] = 1
    else:
        disposed[disposed.index(d)] = 0

In [11]:
ConstDescriptors = {"HeavyAtomCount": Descriptors.HeavyAtomCount,
                    "NumHeteroatoms": Descriptors.NumHeteroatoms,
                    "NumAliphaticHeterocycles": Descriptors.NumAliphaticHeterocycles,
                    "NumHAcceptors": Descriptors.NumHAcceptors}

In [12]:
descriptors = {}
descriptors.update(ConstDescriptors)

In [13]:
def mol_dsc_calc(mols): 
    return DataFrame({k: f(m) for k, f in descriptors.items()} 
             for m in mols)

In [14]:
descriptors_transformer = FunctionTransformer(mol_dsc_calc, validate=False)

In [15]:
X = descriptors_transformer.transform(mols)
X

,HeavyAtomCount,NumHeteroatoms,NumAliphaticHeterocycles,NumHAcceptors
0,91,32,2,16
1,74,28,2,15
2,102,32,1,16
3,85,23,1,12
4,72,26,2,15
...,...,...,...,...
8942,7,3,0,3
8943,18,5,0,2
8944,12,3,0,3
8945,12,3,0,2


In [16]:
X["tox_disposed"] = disposed
X

,HeavyAtomCount,NumHeteroatoms,NumAliphaticHeterocycles,NumHAcceptors,tox_disposed
0,91,32,2,16,1
1,74,28,2,15,1
2,102,32,1,16,1
3,85,23,1,12,1
4,72,26,2,15,1
...,...,...,...,...,...
8942,7,3,0,3,0
8943,18,5,0,2,0
8944,12,3,0,3,1
8945,12,3,0,2,1


Сгенерируем молекулярные отпечатки и превратим их в дескрипторы тоже

In [17]:
def calc_morgan(mols):
    """ генерация молекулярных отпечатков по методу Моргана с радиусом 2
    """
    for_df = []
    for m in mols:
        arr = zeros((1,), dtype='float32')
        DataStructs.ConvertToNumpyArray(AllChem.GetMorganFingerprintAsBitVect(m, 2, 2048), arr)
        for_df.append(arr)
    return DataFrame(for_df)


In [18]:
morgan_transformer = FunctionTransformer(calc_morgan, validate=False)

In [19]:
M = morgan_transformer.transform(mols)
M

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8942,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8943,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8944,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8945,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Объединим дескрипторы в один датафрейм

In [20]:
df = pd.concat([X,M],axis=1)
df

,HeavyAtomCount,NumHeteroatoms,NumAliphaticHeterocycles,NumHAcceptors,tox_disposed,0,1,2,3,4,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,91,32,2,16,1,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,74,28,2,15,1,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,102,32,1,16,1,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,85,23,1,12,1,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,72,26,2,15,1,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8942,7,3,0,3,0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8943,18,5,0,2,0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8944,12,3,0,3,1,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8945,12,3,0,2,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Теперь необходимо привести дескрипторы к единому виду

In [21]:
scaler = StandardScaler()
scaler.fit(df.values)
X = DataFrame(scaler.transform(df.values), index=df.index, columns=df.columns)
X

,HeavyAtomCount,NumHeteroatoms,NumAliphaticHeterocycles,NumHAcceptors,tox_disposed,0,1,2,3,4,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,6.717969,7.038548,1.665081,4.323073,0.691985,-0.05603,1.688634,-0.186264,-0.067014,-0.148875,...,-0.156914,-0.093172,-0.044899,-0.096176,-0.101368,-0.09378,-0.140004,-0.088159,-0.0703,-0.073443
1,5.087186,5.964142,1.665081,3.959904,0.691985,-0.05603,1.688634,-0.186264,-0.067014,-0.148875,...,-0.156914,-0.093172,-0.044899,-0.096176,-0.101368,-0.09378,-0.140004,-0.088159,-0.0703,-0.073443
2,7.773181,7.038548,0.514252,4.323073,0.691985,-0.05603,1.688634,-0.186264,-0.067014,-0.148875,...,-0.156914,-0.093172,-0.044899,-0.096176,-0.101368,-0.09378,-0.140004,-0.088159,-0.0703,-0.073443
3,6.142398,4.621135,0.514252,2.870399,0.691985,-0.05603,1.688634,-0.186264,-0.067014,-0.148875,...,-0.156914,-0.093172,-0.044899,-0.096176,-0.101368,-0.09378,-0.140004,-0.088159,-0.0703,-0.073443
4,4.895329,5.426939,1.665081,3.959904,0.691985,-0.05603,1.688634,-0.186264,-0.067014,-0.148875,...,-0.156914,-0.093172,-0.044899,-0.096176,-0.101368,-0.09378,-0.140004,-0.088159,-0.0703,-0.073443
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8942,-1.340017,-0.750895,-0.636577,-0.398117,-1.445118,-0.05603,1.688634,-0.186264,-0.067014,-0.148875,...,-0.156914,-0.093172,-0.044899,-0.096176,-0.101368,-0.09378,-0.140004,-0.088159,-0.0703,-0.073443
8943,-0.284805,-0.213692,-0.636577,-0.761286,-1.445118,-0.05603,1.688634,-0.186264,-0.067014,-0.148875,...,-0.156914,-0.093172,-0.044899,-0.096176,-0.101368,-0.09378,-0.140004,-0.088159,-0.0703,-0.073443
8944,-0.860375,-0.750895,-0.636577,-0.398117,0.691985,-0.05603,1.688634,-0.186264,-0.067014,-0.148875,...,-0.156914,-0.093172,-0.044899,-0.096176,-0.101368,-0.09378,-0.140004,-0.088159,-0.0703,-0.073443
8945,-0.860375,-0.750895,-0.636577,-0.761286,0.691985,-0.05603,-0.592195,-0.186264,-0.067014,-0.148875,...,-0.156914,-0.093172,-0.044899,-0.096176,-0.101368,-0.09378,-0.140004,-0.088159,-0.0703,-0.073443


Переходим к обучению и подбору моделей

In [22]:
kf = KFold(n_splits=5, random_state=1, shuffle=True)

In [23]:
y = labels

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.2, random_state=0)

Модель 1

In [25]:
parameters = {
    "loss":['deviance'],
    "learning_rate": [0.1,],
    "min_samples_split": [0.2],
    "min_samples_leaf": [0.1],
    "max_depth": [5],
    "max_features": ["sqrt"],
    "criterion": ["friedman_mse"],
    "subsample": [0.8],
    "n_estimators": [700]
    }

clf = GridSearchCV(GradientBoostingClassifier(), parameters, refit=False, cv=2, n_jobs=-1)
clf.fit(X_train, y_train)
print(clf.best_score_)
print(clf.best_params_)

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
/home/alyferryhalo/anaconda3/envs/my-rdkit-env/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/home/alyferryhalo/anaconda3/envs/my-rdkit-env/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/home/alyferryhalo/anaconda3/envs/my-rdkit-env/lib/python3.9/site-packag

0.6962420607384452
{'criterion': 'friedman_mse', 'learning_rate': 0.1, 'loss': 'deviance', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 0.1, 'min_samples_split': 0.2, 'n_estimators': 700, 'subsample': 0.8}


Модель 2

In [ ]:
parameters = {
    "loss":['deviance'],
    "learning_rate": [0.1],
    "min_samples_split": [0.2],
    "min_samples_leaf": [0.1],
    "max_depth": [5],
    "max_features": ["sqrt"],
    "criterion": ["friedman_mse"],
    "subsample": [0.8],
    "n_estimators": [500, 700, 900]
    }

clf = GridSearchCV(GradientBoostingClassifier(), parameters, refit=False, cv=2, n_jobs=-1)
clf.fit(X_train, y_train)
print(clf.best_score_)
print(clf.best_params_)

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: scip

Модель 3

In [ ]:
parameters = {
    "loss":['deviance'],
    "learning_rate": [0.1, 0.05, 0.15, 0.2],
    "min_samples_split": [0.2],
    "min_samples_leaf": [0.1],
    "max_depth": [5],
    "max_features": ["sqrt"],
    "criterion": ["friedman_mse"],
    "subsample": [0.8],
    "n_estimators": [700]
    }

clf = GridSearchCV(GradientBoostingClassifier(), parameters, refit=False, cv=2, n_jobs=-1)
clf.fit(X_train, y_train)
print(clf.best_score_)
print(clf.best_params_)

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: scip

In [32]:
parameters = {'criterion': ['friedman_mse'], 
              'learning_rate': [0.15], 
              'loss': ['exponential'], 
              'max_depth': [5], 
              'max_features': ['sqrt'], 
              'min_samples_leaf': [0.1],
              'min_samples_split': [0.2], 
              'n_estimators': [700], 
              'subsample': [0.8]}

clf = GridSearchCV(GradientBoostingClassifier(), parameters, refit=False, cv=2, n_jobs=-1)
clf.fit(X_train, y_train)
print(clf.best_score_)
print(clf.best_params_)

/home/alyferryhalo/anaconda3/envs/my-rdkit-env/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/home/alyferryhalo/anaconda3/envs/my-rdkit-env/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/home/alyferryhalo/anaconda3/envs/my-rdkit-env/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/home/alyferryhalo/anaconda3/envs/my-rdkit-env/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strin

0.6955444396392783
{'criterion': 'friedman_mse', 'learning_rate': 0.15, 'loss': 'exponential', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 0.1, 'min_samples_split': 0.2, 'n_estimators': 700, 'subsample': 0.8}


In [29]:
parameters = {
    "loss":['exponential'],
    "learning_rate": [0.15],
    "min_samples_split": [3],
    "min_samples_leaf": [0.1],
    "max_depth": [5],
    "max_features": ["sqrt"],
    "criterion": ["friedman_mse"],
    "subsample": [0.8],
    "n_estimators": [700]
    }

clf = GridSearchCV(GradientBoostingClassifier(), parameters, refit=False, cv=2, n_jobs=-1)
clf.fit(X_train, y_train)
print(clf.best_score_)
print(clf.best_params_)

/home/alyferryhalo/anaconda3/envs/my-rdkit-env/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/home/alyferryhalo/anaconda3/envs/my-rdkit-env/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/home/alyferryhalo/anaconda3/envs/my-rdkit-env/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/home/alyferryhalo/anaconda3/envs/my-rdkit-env/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strin

0.6951248596128806
{'criterion': 'friedman_mse', 'learning_rate': 0.15, 'loss': 'exponential', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 0.1, 'min_samples_split': 3, 'n_estimators': 700, 'subsample': 0.8}
